In [ ]:
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# EdgeDriver 路徑
driver_path = r"C:\edgedriver_win64\msedgedriver.exe"  # 使用您的 EdgeDriver 路徑
options = Options()
options.add_argument('--incognito')  # 開啟無痕模式

# 建立 WebDriver
service = Service(driver_path)
driver = webdriver.Edge(service=service, options=options)

# 開啟 Google Earth
url = "https://earth.google.com/web/"
driver.get(url)

# 搜尋欄的 XPath
search_input_xpath = "/html/body/flutter-view/flt-text-editing-host/form[last()]/input[1]"

# 等待搜尋框出現
try:
    search_input = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, search_input_xpath))
    )
    print("搜尋欄已找到，檢查是否可互動...")

    # 強制滾動到搜尋欄並確保可見
    driver.execute_script("arguments[0].scrollIntoView(true);", search_input)
    driver.execute_script("arguments[0].style.visibility='visible';", search_input)

    # 模擬點擊以激活搜尋欄
    driver.execute_script("arguments[0].click();", search_input)
except Exception as e:
    print(f"搜尋欄初始化失敗，錯誤訊息：{e}")
    driver.quit()
    exit()

# 讀取 CSV
file_path = 'C:/Users/USER/cafe/louisa_stores.csv'  # 替換為你的 CSV 檔案路徑
output_path = 'C:/Users/USER/cafe/louisa_position.csv'

print("讀取 CSV 檔案中...")
data = pd.read_csv(file_path)
print(f"讀取完成，共有 {len(data)} 筆資料。")

# 初始化結果列表
coordinates = []

# 搜尋每個地址
for index, row in data.iterrows():
    address = row['地址']  # 假設 CSV 中有 "地址" 欄位
    print(f"正在搜尋第 {index + 1} 筆地址：{address}...")
    try:
        # 等待搜尋框可用
        search_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, search_input_xpath))
        )

        # 確保搜尋欄可互動
        driver.execute_script("arguments[0].scrollIntoView(true);", search_input)
        driver.execute_script("arguments[0].style.visibility='visible';", search_input)
        driver.execute_script("arguments[0].click();", search_input)
        
        # 清除舊內容並輸入地址
        search_input.clear()
        search_input.send_keys(address)
        search_input.send_keys(Keys.RETURN)
        time.sleep(8)  # 等待地圖加載結果

        # 提取座標
        current_url = driver.current_url
        match = re.search(r'@([-0-9.]+),([-0-9.]+),', current_url)
        if match:
            lat, lng = match.groups()
            coordinates.append((address, lat, lng))
            print(f"地址：{address}，座標：({lat}, {lng})")
        else:
            coordinates.append((address, None, None))
            print(f"未能提取地址的座標：{address}")

        time.sleep(5)  # 適當延遲以避免被系統判定為機器人

    except Exception as e:
        print(f"處理地址時發生錯誤：{address}，錯誤訊息：{e}")
        coordinates.append((address, None, None))

# 關閉瀏覽器
driver.quit()

# 保存結果到 CSV
result_df = pd.DataFrame(coordinates, columns=['地址', '緯度', '經度'])
result_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"處理完成，結果已保存至 {output_path}")


In [79]:
search_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, search_input_xpath))
)

# 使用 JavaScript 明確設定焦點
driver.execute_script("arguments[0].focus();", search_input)
search_input.clear()
search_input.send_keys(address)

ElementNotInteractableException: Message: element not interactable
  (Session info: MicrosoftEdge=131.0.2903.146)
Stacktrace:
	(No symbol) [0x00007FF6F97B6B05]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6F9ADF444+1437348]
	(No symbol) [0x00007FF6F969519A]
	(No symbol) [0x00007FF6F96D5336]
	(No symbol) [0x00007FF6F96FC19A]
	(No symbol) [0x00007FF6F96D3437]
	(No symbol) [0x00007FF6F96D32FD]
	(No symbol) [0x00007FF6F96FC480]
	(No symbol) [0x00007FF6F96D3437]
	(No symbol) [0x00007FF6F9716BFF]
	(No symbol) [0x00007FF6F96FBE03]
	(No symbol) [0x00007FF6F96D2984]
	(No symbol) [0x00007FF6F96D1E30]
	(No symbol) [0x00007FF6F96D2571]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6F9A8BAD4+1094964]
	(No symbol) [0x00007FF6F97F32B8]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6F9A8AF13+1091955]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6F9A8AA79+1090777]
	Microsoft::Applications::Events::ILogConfiguration::operator*

In [80]:
is_enabled = search_input.is_enabled()
is_displayed = search_input.is_displayed()
print(f"搜尋欄啟用狀態: {is_enabled}, 顯示狀態: {is_displayed}")

搜尋欄啟用狀態: True, 顯示狀態: False
